In [1]:
import pandas as pd
from simplify_deployment.organism import Organism
import plotly.express as px
from sklearn.linear_model import LinearRegression


In [ ]:
prediction_list = []
for fold in range(12):
    # Raw
    X_train_raw = pd.read_parquet(f"/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/folds/X_train_fold_{fold}.parquet")
    X_test_raw = pd.read_parquet(f"/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/folds/X_test_fold_{fold}.parquet")
    y_train_raw = pd.read_parquet(f"/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/folds/y_train_fold_{fold}.parquet")
    y_test_raw = pd.read_parquet(f"/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/folds/y_test_fold_{fold}.parquet")

    # Data for model
    org = Organism.from_yaml(
        path_config="/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/lag_25_s1_config.yaml",
        path_genome=f"/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/best_genome/lag_25_s1_50_gen_fold_{fold}_best_genome.yaml"
    )
    y_train, X_train = org.create_y_X(
        y_train_raw,
        X_train_raw,
    )
    y_test, X_test = org.create_y_X(
        y_test_raw,
        X_test_raw,
    )

    # Model
    X_train["hour"] = X_train.index.hour
    X_test["hour"] = X_test.index.hour
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction_list.append(
        pd.DataFrame(
            {
                "y_pred_hour_ordinal_encoded": model.predict(X_test),
            },
            index = X_test.index,
        )
    )

    print(f"Fold {fold} done.")

In [ ]:
X_train

In [4]:
prediction = pd.concat(
    prediction_list,
    axis = 0,
)
prediction.to_parquet("/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/hour_ordinal_encoded.parquet")

In [ ]:
fig = px.line(X_train, x = X_train.index, y = list(X_train.columns[:11]))
fig.show()

In [ ]:
prediction